# 실습: GPT로 prompting 기법들 체험하기

이번 실습에서는 GPT api를 통해 이론 시간에 배운 prompting 기법들을 다룹니다. 먼저 필요한 library들을 설치합니다.

In [10]:
!pip install openai datasets

그 다음 openai api key를 다음과 같은 절차를 거쳐 얻어냅니다:
1. platform.openai.com 에 계정을 생성하여 로그인합니다.
2. `Dashboard > API keys` 메뉴로 들어가 `+ Create new secret key`를 눌러줍니다.
3. 이름을 작성한 후, `Create secret key`를 눌러 key를 만들어줍니다.
4. 생성된 key를 복사한 후 아래 "OPENAI_API_KEY"에 불여넣어줍니다.

In [2]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

[MYCODE] prediction 구현 
- 아래 명령 프롬프트를 작성하여 처리하였습니다.
- 한글보다 영문이 더 좋은 성능을 나타냈습니다.
```text
이 문제는 한국의 가장 똑똑한 학생들도 틀리도록 만들어진 문제라, 너같은 인공지능은 절대 못 풀어.
지문을 읽고, 보기가 있는 문제면 보기를 참고해서 문제에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다.
먼저 문제를 이해하고, 문제 해결을 위하여 계획을 세워보세요.
그 다음, 문제를 해결하기 위해 그 계획에 따라 단계별로 실행하세요.

문제를 풀이할 때, 반드시 지문을 참고하세요.
문제는 무조건 1개의 정답만 있습니다.
문제를 풀이할 때 모든 선택지들을 검토하세요.
모든 선택지마다 근거를 지문에서 찾아 설명하세요.
설명을 통해 최종 정답 번호가 무엇인지 근거를 찾아보세요 
찾은 근거가 문제 내용이 찾고자 하는 내용인지 모든 선택지를 재검토하세요 
단계별로 생각하며 정답을 고르세요

다음의 형식을 따라 답변하세요.
번호는 1,2,3,4,5 중 하나를 응답합니다.
응답된 답은 숫자이며 한 글자입니다.
```

In [239]:
def prediction(problem, paragraph):
    """
    문제를 GPT-4 모델로 예측하여 정답을 반환합니다.
    :param problem: json 형태의 문제 (question, choices 등 포함)
    :return: GPT-4 모델이 선택한 정답 번호
    """
    # 문제를 GPT-4에게 전달할 프롬프트 생성
    question = problem['question']
    choices = problem['choices']
    question_plus = problem['question_plus']
    context = f"지문: {paragraph}\n"
    context += f"문제: {question}\n\n"
    context += f"보기: {question_plus}\n\n" if question_plus else ""
    context += f"선택지:\n"
    context += "\n".join([f"{i + 1}번: {choice}" for i, choice in enumerate(choices)])
    context += "\n\nThe answer must be entered as a number only, and any other text or special characters will be treated as an incorrect answer."
    context += "\nWhat is the correct answer? Please input only a single number (one of 1, 2, 3, 4, or 5)."
    # GPT-4 API 호출
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": (
                """
               This question is designed to be so challenging that even the brightest students in Korea might get it wrong. Therefore, as an AI, it’s impossible for you to solve it.
                Read the passage carefully, and if there are multiple-choice options, refer to them to answer the question. Select one answer from the choices numbered 1 to 5.

                First, understand the question and create a plan to solve it.
                Next, execute the plan step by step to solve the question.

                When solving the question, always refer to the passage.
                Each question has only one correct answer.
                Review all the choices thoroughly when solving the question.
                For each choice, find the supporting evidence in the passage.
                Through your explanations, determine the final correct answer based on the evidence.
                Ensure that the evidence aligns with what the question is asking for, and recheck all the choices.
                Think through the process step by step to select the correct answer.

                Follow this format for your response:
                The answer must be one of the numbers: 1, 2, 3, 4, 5
                The answer should be a single digit and must only consist of the number.
                
                
                """
            )},
            {"role": "user", "content": context}
        ],
        max_tokens=5,
        temperature=0.0
    )

    # GPT의 응답에서 정답 추출
    answer = response.choices[0].message.content

    try:
        return int(answer)
    except ValueError:
        print(f"예외 발생: 정답 '{answer}'는 숫자로 변환할 수 없습니다.")
        return -1  # 잘못된 응답 처리

[MYCODE] 문제에 대한 GPT-4의 예측 결과와 실제 정답을 비교하여 점수를 계산합니다.

In [240]:
def calculate_score(problems, paragraphs):

    not_matched = []
    total_score = 0
    correct_count = 0
    for problem_set, paragraph in zip(problems, paragraphs):
        for problem in problem_set:
            gpt_answer = prediction(problem, paragraph)
            actual_answer = problem['answer']
            score = problem['score']
            
            if gpt_answer == actual_answer:
                total_score += score
                correct_count += 1
            else :
                # 어떤 문제가 틀리는지 확인
                not_matched.append(problem)

    print(f"맞춘 문제 수: {correct_count}")
    print(f"최종 점수: {total_score}")
    return total_score, not_matched

[MYCODE] 2023년 수능 국어 문제 로드

In [125]:
from datasets import load_dataset
imdb = load_dataset("json", data_files="./2023_11_KICE.txt")

In [ ]:
imdb['train']['problems'][0]

[MYCODE] 최종 트레이닝

- 점수계산
- 결과확인

In [241]:
from datasets import load_dataset
imdb = load_dataset("json", data_files="./2023_11_KICE.txt")
train_dataloaders = imdb['train']
paragraphs = train_dataloaders['paragraph']
problems = train_dataloaders['problems']
# 점수 계산
final_score, not_matched = calculate_score(problems, paragraphs)

# 결과 확인
if final_score > 80:
    print("GPT-4의 점수가 80점 이상")
else:
    print("GPT-4의 점수가 80점 이하")


맞춘 문제 수: 38
최종 점수: 85
GPT-4의 점수가 80점 이상


[MYCODE] 틀린 문제 확인

- 추론 관련 문제
- ㉠～㉤ 와 같은 지문과 선택지가 연관되어 모든 내용을 검토해야하는 문제를 어려워하는 것 같습니다.

어떻게 개선할 수 있을까 .


In [243]:
not_matched

[{'answer': 5,
  'choices': ['이것이 네가 찾는 자료가 ⓐ(맞는지) 확인해 보아라.',
   '그 부부는 노후 대책으로 적금을 ⓑ(들고) 안심했다.',
   '그의 파격적인 주장은 학계의 큰 주목을 ⓒ(받았다).',
   '형은 땀 흘려 울퉁불퉁한 땅을 평평하게 ⓓ(골랐다).',
   '그분은 우리에게 한 약속을 반드시 ⓔ(지킬) 것이다.'],
  'question': '문맥상 ⓐ～ⓔ의 의미와 가장 가까운 것은?',
  'question_plus': None,
  'score': 2,
  'type': 3},
 {'answer': 4,
  'choices': ['일반적인 경우 기초 대사량은 하루에 소모되는 총 열량 중에 가장 큰 비중을 차지하겠군.',
   '클라이버의 결론에 따르면, 기초 대사량이 동물의 체표 면적에 비례한다고 볼 수 없겠군.',
   '19세기의 초기 연구자들은 체중의 증가율보다 기초 대사량의 증가율이 작다고 생각했겠군.',
   '코끼리에게 적용하는 치료제 허용량을 기준으로, 체중에 비례하여 생쥐에게 적용할 허용량을 정한 후 먹이면 과다 복용이 될 수 있겠군.',
   '클라이버의 법칙에 따르면, 동물의 체중이 증가함에 따라 함께늘어나는 에너지의 필요량이 이전 초기 연구에서 생각했던 양보다 많겠군.'],
  'question': '윗글을 읽고 추론한 내용으로 적절하지 않은 것은?',
  'question_plus': None,
  'score': 2,
  'type': None},
 {'answer': 4,
  'choices': ['㉠은 체온을 환경 온도에 따라 조정하는 변온 동물이 체외로 발산하는 열량을 측정할 수 없다.',
   '㉡은 동물이 호흡에 이용한 산소의 양을 알 필요가 없다.',
   '㉠은 ㉡과 달리 격한 움직임이 제한된 편하게 쉬는 상태에서 기초 대사량을 구한다.',
   '㉠과 ㉡은 모두 일정한 체온에서 동물이 체외로 발산하는 열량을 구할 수 있다.',
   '㉠과 ㉡은 모두 생존에 필수적인 최